# Statements

## Problemt statement
The main problem I'll be tackling is regarding the effects of inflation on restaurants:
```
9. Comparing Public Inflation Data and Sales
-- Objective --
Evaluate how changes in local or national inflation rates (from publicly available data) correlate with sales revenue, average check size, or net revenue. Explore whether venues in regions with higher inflation rates see different spending patterns, such as smaller average bills or lower tip percentages.
```

I will begin by summarizing and making assumptions on the dataset,

# TouchBistro Data Summary

In [47]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import dask.array as nd
from ydata_profiling import ProfileReport


bills = dd.read_csv('../data/CxCData/bills.csv')
venues = dd.read_csv('../data/CxCData/venues.csv')


In [4]:
bills.head()

,bill_paid_at_local,bill_total_billed,bill_total_discount_item_level,bill_total_gratuity,bill_total_net,bill_total_tax,bill_total_voided,bill_uuid,business_date,order_duration_seconds,order_seated_at_local,order_closed_at_local,order_take_out_type_label,order_uuid,payment_amount,payment_count,payment_total_tip,sales_revenue_with_tax,venue_xref_id,waiter_uuid
0,2024-07-01 09:17:01,21.81,0.0,0.0,19.30,2.51,0.0,240701091701~4D63608F-523C-4EFF-9A4F-78D6C44B5159,2024-07-01,46,2024-07-01 09:16:15,2024-07-01 09:17:01,dinein,240701091615~EF1C6E91-B6C4-4DF1-8A92-1B024197FEC3,21.81,1,0.0,21.81,39e5b4830d4d9c14db7368a95b65d5463ea3d095203737...,161027134744~719D0E09-5CA7-442A-B9FA-BBBF4083B4FD
1,2024-07-01 13:37:00,48.95,0.0,0.0,43.32,5.63,0.0,240701133659~3A0BEDE2-E9E5-484D-B909-780E485F0D69,2024-07-01,288,2024-07-01 13:32:12,2024-07-01 13:37:00,dinein,240701133212~A4C33BFA-A54F-4627-B0C2-7428427FB5DE,48.95,1,0.0,48.95,39e5b4830d4d9c14db7368a95b65d5463ea3d095203737...,161027134744~719D0E09-5CA7-442A-B9FA-BBBF4083B4FD
2,2024-07-01 11:19:32,26.94,0.0,0.0,23.84,3.10,0.0,240701111931~670EEB85-E939-4924-B92F-C95076B7E930,2024-07-01,135,2024-07-01 11:17:17,2024-07-01 11:19:32,dinein,240701111717~6074B0AB-2432-484D-BC3F-55CC5D731818,26.94,1,0.0,26.94,39e5b4830d4d9c14db7368a95b65d5463ea3d095203737...,161027134744~719D0E09-5CA7-442A-B9FA-BBBF4083B4FD
3,2024-07-01 15:05:59,3.96,0.0,0.0,3.77,0.19,0.0,240701150558~3D6F473E-D5EF-4BB8-AFAB-30DB2A0764C0,2024-07-01,48,2024-07-01 15:05:11,2024-07-01 15:05:59,dinein,240701150511~EAA68F41-20ED-4FC7-B902-B0B3878AEC82,3.96,1,0.0,3.96,39e5b4830d4d9c14db7368a95b65d5463ea3d095203737...,161027134744~719D0E09-5CA7-442A-B9FA-BBBF4083B4FD
4,2024-07-01 12:41:00,3.36,0.0,0.0,3.20,0.16,0.0,240701124059~0521794C-51FC-4C30-A368-995CE4DE105F,2024-07-01,103,2024-07-01 12:39:17,2024-07-01 12:41:00,dinein,240701123917~27764E1C-7E7F-4293-ABC7-5921310A654A,3.36,1,0.0,3.36,39e5b4830d4d9c14db7368a95b65d5463ea3d095203737...,161027134744~719D0E09-5CA7-442A-B9FA-BBBF4083B4FD


In [3]:
venues.head()

,venue_xref_id,concept,city,country,start_of_day_offset
0,ea74268311cfcc47d2b4c38ef08b9dab0117226a9a99a9...,FAST_FOOD,Albuquerque,US,00:00:00
1,e7cec41c46a9706ba1a702b368be5431913b6dc9c9da49...,FAMILY_DINING,Brooklyn,US,00:00:00
2,e63e79791883b0c4ac71d41eaee727932c13d6a4ec8f78...,FAST_CASUAL,Windsor,CA,00:00:00
3,c100388c9328b30b6bdfbea43f113d367ae9d32b03e84b...,BAR,Burnaby,CA,01:00:00
4,a819ee7f83f27dd3d7f44515e22aca3c26e6704ddd2a9b...,FAMILY_DINING,Tisdale,CA,00:00:00


### Check the general summary of the data


In [4]:
# Here we can view the full report in the browser (importing from teammate since it takes long to process)
import webbrowser

webbrowser.open('../Problem 1/bill_report.html')

True

In [5]:
bills.describe().compute()

,bill_total_billed,bill_total_discount_item_level,bill_total_gratuity,bill_total_net,bill_total_tax,bill_total_voided,order_duration_seconds,payment_amount,payment_count,payment_total_tip,sales_revenue_with_tax
count,8.954786e+06,8.954786e+06,8.954786e+06,8.954786e+06,8.954786e+06,8.954786e+06,8.954786e+06,8.954786e+06,8.954786e+06,8.954786e+06,8.954786e+06
mean,3.674831e+01,8.403802e-01,2.408956e-01,3.389894e+01,2.849375e+00,9.375093e-01,8.790968e+03,4.063878e+01,9.844425e-01,3.654212e+00,3.674902e+01
std,8.337650e+01,9.404221e+00,8.409234e+00,7.754356e+01,7.153385e+00,3.365966e+02,3.984398e+05,1.987292e+02,1.926432e-01,1.753186e+02,8.337668e+01
min,-5.731500e+03,-9.164000e+01,-6.000000e+01,-5.072120e+03,-6.593800e+02,-4.501500e+03,-4.400000e+01,-5.731500e+03,0.000000e+00,-2.531400e+02,-5.731500e+03
25%,1.310000e+01,0.000000e+00,0.000000e+00,1.200000e+01,7.700000e-01,0.000000e+00,1.930000e+02,1.400000e+01,1.000000e+00,0.000000e+00,1.310000e+01
50%,2.709000e+01,0.000000e+00,0.000000e+00,2.500000e+01,1.950000e+00,0.000000e+00,2.021000e+03,2.936000e+01,1.000000e+00,5.000000e-01,2.709000e+01
75%,5.213000e+01,0.000000e+00,0.000000e+00,4.798000e+01,4.420000e+00,0.000000e+00,4.037000e+03,5.760000e+01,1.000000e+00,5.950000e+00,5.213000e+01
max,7.492861e+04,5.490920e+03,1.193553e+04,6.630850e+04,8.620110e+03,1.000000e+06,2.894124e+08,5.227523e+05,4.100000e+01,5.227156e+05,7.492861e+04


In [6]:
venues.describe().compute()

,venue_xref_id,concept,city,country,start_of_day_offset
unique,601,14,387,2,18
count,601,465,601,601,601
top,0002a1cf14e9c1acaa8255fd6777d916d3aec6bc1f3c8a...,FAMILY_DINING,Toronto,CA,00:00:00
freq,1,173,52,334,401


### We check for invalid data

In [7]:
#  NaN values
bills.isnull().sum().compute()

bill_paid_at_local                 0
bill_total_billed                  0
bill_total_discount_item_level     0
bill_total_gratuity                0
bill_total_net                     0
bill_total_tax                     0
bill_total_voided                  0
bill_uuid                          0
business_date                      0
order_duration_seconds             0
order_seated_at_local              0
order_closed_at_local              0
order_take_out_type_label          0
order_uuid                         0
payment_amount                     0
payment_count                      0
payment_total_tip                  0
sales_revenue_with_tax             0
venue_xref_id                      0
waiter_uuid                       94
dtype: int64

# Assumptions
We look more in-depth at the characteristics of the data and make some assumptions

In [8]:
bills.loc[bills['sales_revenue_with_tax'] < 0].head()

,bill_paid_at_local,bill_total_billed,bill_total_discount_item_level,bill_total_gratuity,bill_total_net,bill_total_tax,bill_total_voided,bill_uuid,business_date,order_duration_seconds,order_seated_at_local,order_closed_at_local,order_take_out_type_label,order_uuid,payment_amount,payment_count,payment_total_tip,sales_revenue_with_tax,venue_xref_id,waiter_uuid
121,2024-07-13 09:19:43,-7.36,0.0,0.0,-6.51,-0.85,0.0,240713091943~6B7448D4-582B-4658-834F-A81E22720B18,2024-07-13,49,2024-07-13 09:18:54,2024-07-13 09:19:43,bartab,240713091854~4D898EF4-6738-4066-90AD-FF2E4F6D47B3,-7.36,2,0.0,-7.36,39e5b4830d4d9c14db7368a95b65d5463ea3d095203737...,161027134744~719D0E09-5CA7-442A-B9FA-BBBF4083B4FD
504,2024-08-24 15:19:16,-10.18,0.0,0.0,-9.01,-1.17,0.0,240824151915~8B5A5CAA-C3D0-4990-B609-23F38C9B31CF,2024-08-24,11,2024-08-24 15:19:05,2024-08-24 15:19:16,dinein,240824151905~75046DB6-A645-4985-8209-5927844976CC,-10.18,1,0.0,-10.18,39e5b4830d4d9c14db7368a95b65d5463ea3d095203737...,161027134744~719D0E09-5CA7-442A-B9FA-BBBF4083B4FD
809,2024-09-20 16:02:25,-4.04,0.0,0.0,-3.85,-0.19,0.0,240920160225~5E1EB977-7EF7-44C6-B3C3-D3663D93C6A2,2024-09-20,27,2024-09-20 16:01:58,2024-09-20 16:02:25,dinein,240920160158~25F9E252-FD76-4388-9D2C-1B6ECC8BA049,-4.04,1,0.0,-4.04,39e5b4830d4d9c14db7368a95b65d5463ea3d095203737...,161027134744~719D0E09-5CA7-442A-B9FA-BBBF4083B4FD
12918,2024-09-13 14:10:09,-2.50,0.0,0.0,-2.50,0.00,0.0,240913141008~3483EF8B-4FF7-428D-9E12-47CB139DADC6,2024-09-13,105,2024-09-13 14:08:24,2024-09-13 14:10:09,bartab,240913140824~9B111861-DD6A-4730-AC56-D9975FB7C2D6,-2.50,2,0.0,-2.50,9989a37538faecc495bbe774b30fb447a1582b8f1bc14a...,150101000000~9E797F58-9C8D-4B02-9810-82F4FA5785B4
13642,2024-09-05 14:16:05,-7.49,0.0,0.0,-7.49,0.00,0.0,240905141603~DBD662DB-B5FF-4077-82B6-92E4A5BD837F,2024-09-05,20,2024-09-05 14:15:45,2024-09-05 14:16:05,bartab,240905141545~A080CA7E-359E-41E5-82EF-5B05C9A093B3,-7.49,1,0.0,-7.49,9989a37538faecc495bbe774b30fb447a1582b8f1bc14a...,150101000000~9E797F58-9C8D-4B02-9810-82F4FA5785B4


In [9]:
bills.loc[bills['bill_total_voided'] > 0].head()

,bill_paid_at_local,bill_total_billed,bill_total_discount_item_level,bill_total_gratuity,bill_total_net,bill_total_tax,bill_total_voided,bill_uuid,business_date,order_duration_seconds,order_seated_at_local,order_closed_at_local,order_take_out_type_label,order_uuid,payment_amount,payment_count,payment_total_tip,sales_revenue_with_tax,venue_xref_id,waiter_uuid
218,2024-07-25 07:52:00,0.00,0.00,0.0,0.00,0.00,6.15,240725075159~A20499FF-BD04-42AB-B28F-D7431445ACF2,2024-07-25,56706,2024-07-24 16:06:54,2024-07-25 07:52:00,dinein,240724160654~82DC772D-5779-4CD7-83BF-7C2EEEBAAAC6,0.00,0,0.0,0.00,39e5b4830d4d9c14db7368a95b65d5463ea3d095203737...,160817145755~908A3915-8B83-4471-8811-773D0D28FDBE
1216,2024-07-15 07:59:01,5.71,0.00,0.0,5.05,0.66,3.20,240715075900~493533E4-EA6D-49D1-AA41-FE890D9DAF58,2024-07-15,141051,2024-07-13 16:48:10,2024-07-15 07:59:01,dinein,240713164810~B7E8EBFC-2CF2-4911-A80A-6F501FE30EA1,5.71,1,0.0,5.71,39e5b4830d4d9c14db7368a95b65d5463ea3d095203737...,160817145755~908A3915-8B83-4471-8811-773D0D28FDBE
1489,2024-07-29 08:08:04,18.45,0.00,0.0,16.33,2.12,3.75,240729080803~1AD5D17D-9EAC-462B-ABE4-7C9025EA8A83,2024-07-29,142,2024-07-29 08:05:42,2024-07-29 08:08:04,dinein,240729080542~85E612F2-D388-49F7-9B7A-42ADB60ED26E,18.45,1,0.0,18.45,39e5b4830d4d9c14db7368a95b65d5463ea3d095203737...,160817145755~908A3915-8B83-4471-8811-773D0D28FDBE
2185,2024-07-18 12:39:34,8.55,3.75,0.0,7.57,0.98,4.35,240718123933~4B52E48B-69C3-45F1-BA0D-1B31B8131633,2024-07-18,184,2024-07-18 12:36:30,2024-07-18 12:39:34,dinein,240718123630~CFF915C6-2EAA-447A-8636-4F14F77867F5,8.55,1,0.0,8.55,39e5b4830d4d9c14db7368a95b65d5463ea3d095203737...,160817145755~908A3915-8B83-4471-8811-773D0D28FDBE
2284,2024-07-28 14:51:34,13.01,0.00,0.0,11.51,1.50,12.47,240728145132~623C9699-B58F-4282-86F5-AFBECBA02BBA,2024-07-28,375,2024-07-28 14:45:19,2024-07-28 14:51:34,dinein,240728144519~2AA19691-1F64-4EE8-8811-040A8DA43244,13.01,1,0.0,13.01,39e5b4830d4d9c14db7368a95b65d5463ea3d095203737...,160817145755~908A3915-8B83-4471-8811-773D0D28FDBE


In [10]:
labels = ['sales_revenue_with_tax', 'bill_total_voided']

for label in labels:
    neg_count = (bills[label] < 0).sum().compute()
    print(f'--{label} values--\n neg: {neg_count} \n')



--sales_revenue_with_tax values--
 neg: 13817 

--bill_total_voided values--
 neg: 624 



Pairing this with what we saw in the report

**-- sales_revenue_with_tax --**
- Most sales are positive, however it is sometimes negative due to refunds
Looking at the report, we can see the same characteristics are also seen for related columns (`bill_total_billed`, `bill_total_net`), and therefore assume the same.

**-- bill_total_voided --**
- Most voided bills are zero, however it is sometimes positive due to voided, and negative when this is is to be reserved; likely due to miscommunication
Looking at the report, we can see the same characteristics are also seen for related columns (`bill_total_discount_item_level`, `bill_total_gratuity`), and therefore assume the same.


Interestingly, in the report, `bill_total_tax` has a relatively high amount of zeros (14.8%). At first, I assume this may be due to there being no taxes during the winter holiday season (for Canadian restaurants):

In [11]:
bills.loc[bills['bill_total_tax'] <= 0].head()

,bill_paid_at_local,bill_total_billed,bill_total_discount_item_level,bill_total_gratuity,bill_total_net,bill_total_tax,bill_total_voided,bill_uuid,business_date,order_duration_seconds,order_seated_at_local,order_closed_at_local,order_take_out_type_label,order_uuid,payment_amount,payment_count,payment_total_tip,sales_revenue_with_tax,venue_xref_id,waiter_uuid
9,2024-07-01 16:20:10,19.00,0.0,0.0,19.00,0.0,0.0,240701162010~01A577B5-25CB-4908-AF54-73B5428FD83B,2024-07-01,716,2024-07-01 16:08:14,2024-07-01 16:20:10,dinein,240701160814~B6D53396-87A5-4AE9-A83B-B1A059AE4A83,25.00,1,6.0,19.00,39e5b4830d4d9c14db7368a95b65d5463ea3d095203737...,161027134744~719D0E09-5CA7-442A-B9FA-BBBF4083B4FD
46,2024-07-06 12:43:38,18.95,0.0,0.0,18.95,0.0,0.0,240706124337~182679FF-FED1-4A81-958F-13914D99CCAF,2024-07-06,59,2024-07-06 12:42:39,2024-07-06 12:43:38,dinein,240706124239~0B1CF58F-1C18-45EC-8413-7E7F12DACE66,18.95,1,0.0,18.95,39e5b4830d4d9c14db7368a95b65d5463ea3d095203737...,161027134744~719D0E09-5CA7-442A-B9FA-BBBF4083B4FD
50,2024-07-06 14:10:30,24.95,0.0,0.0,24.95,0.0,0.0,240706141029~E2F16C10-A241-47C0-A2A2-4B7BBB0F8601,2024-07-06,41,2024-07-06 14:09:49,2024-07-06 14:10:30,dinein,240706140949~A7A484E9-7513-4828-A7EC-F896EC499CED,24.95,1,0.0,24.95,39e5b4830d4d9c14db7368a95b65d5463ea3d095203737...,161027134744~719D0E09-5CA7-442A-B9FA-BBBF4083B4FD
63,2024-07-07 10:56:32,1.00,0.0,0.0,1.00,0.0,0.0,240707105632~72430C8C-9B84-45CC-BDED-48E9008066CE,2024-07-07,275,2024-07-07 10:51:57,2024-07-07 10:56:32,dinein,240707105157~62323ABF-FAB4-46BF-ABFB-89FE6860F40D,1.00,1,0.0,1.00,39e5b4830d4d9c14db7368a95b65d5463ea3d095203737...,161027134744~719D0E09-5CA7-442A-B9FA-BBBF4083B4FD
77,2024-07-08 15:21:06,24.95,0.0,0.0,24.95,0.0,0.0,240708152105~45AE782D-3129-459F-8133-AC54F068A499,2024-07-08,79,2024-07-08 15:19:47,2024-07-08 15:21:06,dinein,240708151947~7DBEABD3-65A0-4435-BF78-97A2C3060D11,24.95,1,0.0,24.95,39e5b4830d4d9c14db7368a95b65d5463ea3d095203737...,161027134744~719D0E09-5CA7-442A-B9FA-BBBF4083B4FD


In [12]:
bills.loc[bills['bill_total_tax'] <= 0].head()

,bill_paid_at_local,bill_total_billed,bill_total_discount_item_level,bill_total_gratuity,bill_total_net,bill_total_tax,bill_total_voided,bill_uuid,business_date,order_duration_seconds,order_seated_at_local,order_closed_at_local,order_take_out_type_label,order_uuid,payment_amount,payment_count,payment_total_tip,sales_revenue_with_tax,venue_xref_id,waiter_uuid
9,2024-07-01 16:20:10,19.00,0.0,0.0,19.00,0.0,0.0,240701162010~01A577B5-25CB-4908-AF54-73B5428FD83B,2024-07-01,716,2024-07-01 16:08:14,2024-07-01 16:20:10,dinein,240701160814~B6D53396-87A5-4AE9-A83B-B1A059AE4A83,25.00,1,6.0,19.00,39e5b4830d4d9c14db7368a95b65d5463ea3d095203737...,161027134744~719D0E09-5CA7-442A-B9FA-BBBF4083B4FD
46,2024-07-06 12:43:38,18.95,0.0,0.0,18.95,0.0,0.0,240706124337~182679FF-FED1-4A81-958F-13914D99CCAF,2024-07-06,59,2024-07-06 12:42:39,2024-07-06 12:43:38,dinein,240706124239~0B1CF58F-1C18-45EC-8413-7E7F12DACE66,18.95,1,0.0,18.95,39e5b4830d4d9c14db7368a95b65d5463ea3d095203737...,161027134744~719D0E09-5CA7-442A-B9FA-BBBF4083B4FD
50,2024-07-06 14:10:30,24.95,0.0,0.0,24.95,0.0,0.0,240706141029~E2F16C10-A241-47C0-A2A2-4B7BBB0F8601,2024-07-06,41,2024-07-06 14:09:49,2024-07-06 14:10:30,dinein,240706140949~A7A484E9-7513-4828-A7EC-F896EC499CED,24.95,1,0.0,24.95,39e5b4830d4d9c14db7368a95b65d5463ea3d095203737...,161027134744~719D0E09-5CA7-442A-B9FA-BBBF4083B4FD
63,2024-07-07 10:56:32,1.00,0.0,0.0,1.00,0.0,0.0,240707105632~72430C8C-9B84-45CC-BDED-48E9008066CE,2024-07-07,275,2024-07-07 10:51:57,2024-07-07 10:56:32,dinein,240707105157~62323ABF-FAB4-46BF-ABFB-89FE6860F40D,1.00,1,0.0,1.00,39e5b4830d4d9c14db7368a95b65d5463ea3d095203737...,161027134744~719D0E09-5CA7-442A-B9FA-BBBF4083B4FD
77,2024-07-08 15:21:06,24.95,0.0,0.0,24.95,0.0,0.0,240708152105~45AE782D-3129-459F-8133-AC54F068A499,2024-07-08,79,2024-07-08 15:19:47,2024-07-08 15:21:06,dinein,240708151947~7DBEABD3-65A0-4435-BF78-97A2C3060D11,24.95,1,0.0,24.95,39e5b4830d4d9c14db7368a95b65d5463ea3d095203737...,161027134744~719D0E09-5CA7-442A-B9FA-BBBF4083B4FD


However, some of the order dates shown above are not during the winter holiday season, which would be when Canada has its tax free policy for restaurants.
This leads me to think that this is due to tax being included, however this is a toggable setting (https://cdn.touchbistro.com/help/articles/tax-settings/), which means it should be accounted for here.

The only assumption I can make here is that there was some kind of fault when recording the data for it being tax inclusive, or that 

Another note I want to make on `bill_total_tax` is the possibility of tax evasion by under reporting on transactions.

# Cleaning Data

As we are observing the macro side of things, we mostly want the underlying trend rather than every detail.
As a result, I will have a clean dataset with only normal transactions (no voided transactions, no negative tax or discount. Negative sales, net bill, and discount are needed since they're simply refunds and would otherwise overestimate the total if not included)

In [48]:
statement = (
    (bills['bill_total_tax'] > 0) &   # Proper Tax
    (bills['bill_total_voided'] != 0) &  # Exclude voided transactions
    (bills['bill_total_discount_item_level'] >= 0)  # Exclude negative discounts (manual discount fraud prevention)
)

filtered_bills = bills.loc[statement]


We are not interested in the specific UIDs, so we will also drop these columns

In [49]:
filtered_nonunique_bills = filtered_bills.drop(['bill_uuid','order_uuid','waiter_uuid'], axis=1)

We are also interested in only the `concept` (restaurant type), `city`, `country` columns from the venue dataset, so we will also include this

In [50]:
venue_bills = filtered_nonunique_bills.merge(right=venues, how='left', left_on='venue_xref_id', right_on='venue_xref_id')
filterd_venue_bills = venue_bills.drop(['venue_xref_id','start_of_day_offset'], axis=1)

In [51]:
filterd_venue_bills.head()

,bill_paid_at_local,bill_total_billed,bill_total_discount_item_level,bill_total_gratuity,bill_total_net,bill_total_tax,bill_total_voided,business_date,order_duration_seconds,order_seated_at_local,order_closed_at_local,order_take_out_type_label,payment_amount,payment_count,payment_total_tip,sales_revenue_with_tax,concept,city,country
0,2024-07-15 07:59:01,5.71,0.00,0.0,5.05,0.66,3.20,2024-07-15,141051,2024-07-13 16:48:10,2024-07-15 07:59:01,dinein,5.71,1,0.0,5.71,<NA>,Collingwood,CA
1,2024-07-29 08:08:04,18.45,0.00,0.0,16.33,2.12,3.75,2024-07-29,142,2024-07-29 08:05:42,2024-07-29 08:08:04,dinein,18.45,1,0.0,18.45,<NA>,Collingwood,CA
2,2024-07-18 12:39:34,8.55,3.75,0.0,7.57,0.98,4.35,2024-07-18,184,2024-07-18 12:36:30,2024-07-18 12:39:34,dinein,8.55,1,0.0,8.55,<NA>,Collingwood,CA
3,2024-07-28 14:51:34,13.01,0.00,0.0,11.51,1.50,12.47,2024-07-28,375,2024-07-28 14:45:19,2024-07-28 14:51:34,dinein,13.01,1,0.0,13.01,<NA>,Collingwood,CA
4,2024-07-26 14:09:40,15.37,0.00,0.0,13.60,1.77,7.82,2024-07-26,17618,2024-07-26 09:16:02,2024-07-26 14:09:40,dinein,15.37,1,0.0,15.37,<NA>,Collingwood,CA


In [52]:
filterd_venue_bills.compute().to_csv('normal_bills.csv', index=False)